In [1]:
# Dependencies and setup of modules for ETL Project
import pandas as pd
import numpy as np
import unidecode
from sqlalchemy import create_engine

# Extract CSV files into DataFrames

In [2]:
# DC airbnb file
dc_airbnb_file = "Data/DC_airbnb_listings.csv"
# DC hotel file
dc_hotel_file = "Data/DC_Hotels.csv"
# DC airbnb dataframe
dc_airbnb_df = pd.read_csv(dc_airbnb_file, index_col="id", encoding="utf-8")
# DC hotel dataframe
dc_hotel_df = pd.read_csv(dc_hotel_file, index_col="objectid", encoding="utf-8")
#dc_airbnb_df.head()
#dc_hotel_df.head()

/anaconda3/envs/PythonData/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# DC airbnb columns
dc_airbnb_df.columns
#dc_hotel_df.columns

Index(['listing_url', 'scrape_id', 'last_scraped', 'name', 'summary', 'space',
       'description', 'experiences_offered', 'neighborhood_overview', 'notes',
       ...
       'instant_bookable', 'is_business_travel_ready', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month'],
      dtype='object', length=105)

# DC Airbnb data analysis and cleanup

In [4]:
#dc_airbnb_cols = ["name", "summary", "transit", "access", "host_name", "host_location", "host_neighbourhood", "street", "city", "state", "zipcode", "market", "smart_location", "country_code", "country", "latitude", "longitude", "property_type", "room_type"]

In [5]:
dc_airbnb_df = dc_airbnb_df[["name","summary", "transit", "access", "host_name", "host_location",\
                             "host_neighbourhood", "street", "city", "state", "zipcode", "market",\
                             "smart_location", "country_code", "country", "latitude", "longitude", \
                             "property_type", "room_type"]]
# Since columns summary, transit, access, name, host_name, host_location had bad data and not required,hence dropped
# new clean dataframe created
dc_airbnb_df_clean = dc_airbnb_df.drop(columns=['summary','transit','access','name','host_name','host_location'])
dc_airbnb_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9001 entries, 3362 to 32284361
Data columns (total 13 columns):
host_neighbourhood    7954 non-null object
street                9001 non-null object
city                  8998 non-null object
state                 8989 non-null object
zipcode               8839 non-null object
market                8976 non-null object
smart_location        9001 non-null object
country_code          9001 non-null object
country               9001 non-null object
latitude              9001 non-null float64
longitude             9001 non-null float64
property_type         9001 non-null object
room_type             9001 non-null object
dtypes: float64(2), object(11)
memory usage: 984.5+ KB


In [6]:
# Zip code is the key to join dc airbnb and dc hotels, there are few Nan data, hence are cleaned up
dc_airbnb_df_clean = dc_airbnb_df_clean.dropna(subset=['zipcode'])
dc_airbnb_df_clean.info()
#dc_airbnb_df_clean.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8839 entries, 3362 to 32284361
Data columns (total 13 columns):
host_neighbourhood    7866 non-null object
street                8839 non-null object
city                  8838 non-null object
state                 8830 non-null object
zipcode               8839 non-null object
market                8814 non-null object
smart_location        8839 non-null object
country_code          8839 non-null object
country               8839 non-null object
latitude              8839 non-null float64
longitude             8839 non-null float64
property_type         8839 non-null object
room_type             8839 non-null object
dtypes: float64(2), object(11)
memory usage: 966.8+ KB


# DC Hotel data analysis and clean up

In [7]:
# DC hotel data information, zip code data is important so it can be joined to the dc airbnb data.
# There are no null values in the data, no clean up required. Other columns are added to table and set default values
dc_hotel_df.info()
#dc_hotel_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 169 entries, 1 to 100
Data columns (total 10 columns):
web_url     167 non-null object
phone       169 non-null object
zip         169 non-null int64
gis_id      132 non-null object
address     169 non-null object
numrooms    141 non-null float64
name        168 non-null object
x           169 non-null float64
y           169 non-null float64
addrid      169 non-null int64
dtypes: float64(3), int64(2), object(5)
memory usage: 14.5+ KB


# Create Database Connection

In [8]:
# Set connection to MySql data base and invoke engine
connection_string = "root:Srini@786@localhost/etl"
engine = create_engine(f'mysql+pymysql://{connection_string}')
# Find what tables exist as they were created before
engine.table_names()

['dc_airbnb', 'dc_hotels']

# Load DataFrame DC Airbnb data into Database table

In [9]:
# Load dataframe dc airbnb in to etl database and table dc_airbnb
dc_airbnb_df_clean.to_sql(name='dc_airbnb', con=engine, if_exists='append', index=True)

# Load DataFrame DC Hotel data into Database table

In [11]:
# Load dataframe dc airbnb in to etl database and table dc_airbnb
dc_hotel_df.to_sql(name='dc_hotels', con=engine, if_exists='append', index=True)